Less of a unit test, more an exploration of waffles.

In [98]:
filename = 'samples/waffle.tex'

In [99]:
# debug
from importlib import reload
import TexSoup
reload(TexSoup)

import TexSoup
import re
import numpy as np

from TexSoup import preprocessing, postprocess

In [100]:
with open(filename,'r') as f:
    tex_doc = f.read()

Sometimes there are weird errors like spaces in begins/ends -- fix these first:

In [102]:
tex_doc = preprocessing.process_begin_end(tex_doc)

Clean accents in case folks did something funny:

In [103]:
tex_doc, error_accents = preprocessing.clean_accents(tex_doc)

In [104]:
#print(tex_doc)

Get new commands and enviroments -- we need to these to check if there are `\begin` and `\end` in these, so first, get them!

In [105]:
newcommands, newenvironments = preprocessing.get_newcommands_and_newenvs(tex_doc,
                                                                         verbose=True)

Get the argument numbers for these -- will need this for search-replace:

In [106]:
args_newcommands = preprocessing.find_args_newcommands(newcommands,error_out=True, verbose=True)

In [107]:
args_newcommands

[['\\be', '\\newcommand{\\be}{\\begin{equation}}', 57, 91, 0],
 ['\\ee', '\\newcommand{\\ee}{\\end{equation}}', 92, 124, 0],
 ['\\bee',
  '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
  126,
  176,
  1],
 ['\\eee', '\\newcommand{\\eee}{\\end{equation}}', 177, 210, 0],
 ['\\beee',
  '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
  212,
  266,
  2],
 ['\\eeee', '\\newcommand{\\eeee}{\\end{equation}}', 267, 301, 0],
 ['\\etal ', '\\def \\etal {{\\em et al.}}', 304, 329, 0]]

In [108]:
args_newenvironments = preprocessing.find_args_newenvironments(newenvironments,
                                                               error_out=True, verbose=True)

In [109]:
args_newenvironments

[]

In [110]:
comments, find_replace, error = preprocessing.generate_find_replace_newcommands(args_newcommands, 
                                                               verbose=True,arg_type = 'newcommand')

Replace newcommands and environments:

In [111]:
tex_doc_nc, error, warnings = preprocessing.replace_newcommands_and_newenvironments(tex_doc, 
                                                                          args_newcommands,
                                                                          args_newenvironments,
                                                                          verbose=True)

args for: \newcommand{\bee}[1]{\begin{equation}\label{e:#1}}
{1: 'lenseq'}
args for: \newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2}
{1: 'lenseq2', 2: 'A(x)=5'}

\newcommand{\be}{\begin{equation}} gets commented
\newcommand{\ee}{\end{equation}} gets commented
\newcommand{\bee}[1]{\begin{equation}\label{e:#1}} gets commented
\newcommand{\eee}{\end{equation}} gets commented
\newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2} gets commented
\newcommand{\eeee}{\end{equation}} gets commented


In [112]:
soup = TexSoup.TexSoup(r''+tex_doc_nc, tolerance=0)

In [114]:
soup_clean = postprocess.clean_slash_commands(soup)

In [115]:
for iss,s in enumerate(soup_clean.all):
    print('----------- soup element =', iss, '---------------')
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                print('----------- sub-soup element =', isss, '---------------')
                #print(type(ss))
                print(type(ss.expr))
                print(ss)
                print('-----------------')
                if str(ss).strip() == '\\': # 
                    import sys; sys.exit()
        else:
            print(' ******** not in document *********** ')

----------- soup element = 0 ---------------
----------- soup element = 1 ---------------
----------- soup element = 2 ---------------
----------- soup element = 3 ---------------
----------- soup element = 4 ---------------
----------- soup element = 5 ---------------
----------- soup element = 6 ---------------
----------- soup element = 7 ---------------
----------- soup element = 8 ---------------
----------- soup element = 9 ---------------
----------- soup element = 10 ---------------
----------- soup element = 11 ---------------
----------- soup element = 12 ---------------
----------- soup element = 13 ---------------
----------- soup element = 14 ---------------
----------- soup element = 15 ---------------
----------- soup element = 16 ---------------
----------- soup element = 17 ---------------
----------- soup element = 18 ---------------
----------- soup element = 19 ---------------
----------- soup element = 20 ---------------
----------- soup element = 21 --------------

In [134]:
texout_splits = postprocess.parse_soup_to_tags(soup,tex_doc_nc, verbose=False)

In [135]:
texout_splits

[('\\documentclass[a4paper]{article}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('\\usepackage{hyperref}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\be}{\\begin{equation}}', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\ee}{\\end{equation}}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\eee}{\\end{equation}}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}', 'others'),
 ('\n', 'others'),
 ('%\\newcommand{\\eeee}{\\end{equation}}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('\\def \\etal {{\\em et al.}}', 'others'),
 ('\n', 'others'),
 ('\n', 'others'),
 ('\\begin{document}\n', 'beginDoc'),
 ('\n', 'whitespace'),
 ('\n', 'whitespace'),
 ('\\section{Waffles waffles waffles}', 'commandOrBracket'),
 ('\n', 'whitespace'),
 ('\n', 'whitespace'),
 ('\\s